# What happens:

- sql is almost done
- all with foor loop
- add the coloring in the header and total

### Import 

In [384]:
import os
import sys
import pandas as pd
import numpy as np
from pandas import DataFrame
import sqlalchemy
from sqlalchemy import create_engine
import getpass
import datetime
import win32com.client as win32
import calendar
from openpyxl import Workbook, load_workbook
from IPython.lib.display import Audio
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.cell import Cell
from openpyxl.styles import Color, Font, PatternFill

### Select the folder where you wanna do the work
### Also ask for the Reporting Date

In [385]:
directory = os.path.abspath(os.getcwd()) + "/"

rd = str(input("Please enter the RD (YYYYMMDD): "))

ws = "0002"

### Make connection with Oracle DB

In [386]:
#Setting up RAY connection:
connection_RAY = create_engine(str("oracle://:@OCRPN"))
connection_RAY.execute("call pack_context.context_open(to_date('{reporting_date}','YYYYMMDD'),{partition})".format(
                 reporting_date = rd, partition = ws))

### Open excel file

In [387]:
im_folder = "Import/"
# determining the name of the file
file_name = 'ExAnte Notification ' + calendar.month_name[int(rd[4:6])] + ' ' + rd[0:4] + '.xlsx'

file = load_workbook(im_folder + "ExAnte Template.xlsx")

sqls = file['sql and workspace']


### Select SQL 1

In [388]:
sql1 = file['sql 1']

d1 = pd.read_sql_query("""select 
case when table_name = 'FUND' and contract_type ='IS2700'   then' Investment in FUND' else rating_model_new  end as rating_model_new_mapped
,rating_model_new,
sum (ead) ead ,sum (rwa) rwa
from rzb_cdr 
where record_source='T_CDR' and method_new = 'FOU' 
and NOT (table_name='EQUITY' and sub_method_new='STD')---to remove the grandfathering equities
group by case when table_name = 'FUND' and contract_type ='IS2700'   then' Investment in FUND' else rating_model_new  end, rating_model_new""", connection_RAY)

### Input the values and add the colors

In [389]:
# add headers
for idx_c, header in enumerate(d1.columns.values):
    sql1.cell(row=1, column=idx_c+1).value = header
    sql1.cell(row=1, column=idx_c+1).fill = PatternFill(start_color='D9E1F2', end_color='D9E1F2', fill_type='solid')

# Iterate all rows using DataFrame.itertuples()
index = 0
for idx, row in enumerate(d1.itertuples(index = True)):
    for idx_c, header in enumerate(d1.columns.values):
        sql1.cell(row=idx+2, column=idx_c+1).value = getattr(row, header)
        index = idx


# Add the total        
sql1.cell(row=index+3, column=2).value = 'Total:'
sql1.cell(row=index+3, column=3).value = '=SUM(C2:C'+str(index+2)+')'
sql1_total = str(index+3)
sql1.cell(row=index+3, column=4).value = '=SUM(D2:D'+str(index+2)+')'

for col in range(2, 5, 1):
    sql1.cell(row=index+3, column=col).fill = PatternFill(start_color='D9E1F2', end_color='D9E1F2', fill_type='solid')


### Select SQL 2

In [390]:
sql2 = file['sql 2']

d2 = pd.read_sql_query("""select
case when table_name = 'FUND' and contract_type ='IS2700' then 'Investment in FUND' else  counterparty_new_type end as counterparty_new_type_mapped,
counterparty_new_type,counterparty_new_type_desc,sum (ead) ead, sum (rwa) rwa from rzb_cdr
where record_source='T_CDR' and method_new = 'FOU'
and NOT (table_name='EQUITY' and sub_method_new='STD')---to remove the grandfathering equities
group by case when table_name = 'FUND' and contract_type ='IS2700' then 'Investment in FUND' else  counterparty_new_type end,counterparty_new_type,counterparty_new_type_desc
""", connection_RAY)

### Input the values and add the colors

In [391]:
# add headers
for idx_c, header in enumerate(d2.columns.values):
    sql2.cell(row=1, column=idx_c+1).value = header
    sql2.cell(row=1, column=idx_c+1).fill = PatternFill(start_color='D9E1F2', end_color='D9E1F2', fill_type='solid')

# Iterate all rows using DataFrame.itertuples()
index = 0
for idx, row in enumerate(d2.itertuples(index = True)):
    for idx_c, header in enumerate(d2.columns.values):
        sql2.cell(row=idx+2, column=idx_c+1).value = getattr(row, header)
        index = idx

# Add the total        
sql2.cell(row=index+3, column=3).value = 'Total:'
sql2.cell(row=index+3, column=4).value = '=SUM(D2:D'+str(index+2)+')'
sql2.cell(row=index+3, column=5).value = '=SUM(E2:E'+str(index+2)+')'

for col in range(3, 6, 1):
    sql2.cell(row=index+3, column=col).fill = PatternFill(start_color='D9E1F2', end_color='D9E1F2', fill_type='solid')
    
sql2.cell(row=index+6, column=3).value = 'check FOU'
sql2.cell(row=index+6, column=4).value = "=D"+str(index+3)+"-'sql 1'!C" + str(sql1_total)
sql2.cell(row=index+6, column=5).value = "=E"+str(index+3)+"-'sql 1'!D" + str(sql1_total)
    

for col in range(3, 6, 1):
    sql2.cell(row=index+6, column=col).fill = PatternFill(start_color='7994D1', end_color='7994D1', fill_type='solid')

### Select SQL 3

In [392]:
sql3 = file['sql 3']

d3 = pd.read_sql_query("""select counterparty_new_category, sum(ead) ead, sum(rwa) rwa from v_rzb_cdr_full where record_source = 'RETAIL' and counterparty_new_category in ('PI','MICRO') group by counterparty_new_category""", connection_RAY)

### Input the values and add the colors

In [393]:
# add headers
for idx_c, header in enumerate(d3.columns.values):
    sql3.cell(row=1, column=idx_c+1).value = header
    sql3.cell(row=1, column=idx_c+1).fill = PatternFill(start_color='D9E1F2', end_color='D9E1F2', fill_type='solid')

# Iterate all rows using DataFrame.itertuples()
index = 0
for idx, row in enumerate(d3.itertuples(index = True)):
    for idx_c, header in enumerate(d3.columns.values):
        sql3.cell(row=idx+2, column=idx_c+1).value = getattr(row, header)
        index = idx

# Add the total        
sql3.cell(row=index+3, column=1).value = 'Total:'
sql3.cell(row=index+3, column=2).value = '=SUM(B2:B'+str(index+2)+')'
sql3.cell(row=index+3, column=3).value = '=SUM(C2:C'+str(index+2)+')'

for col in range(1, 4, 1):
    sql3.cell(row=index+3, column=col).fill = PatternFill(start_color='D9E1F2', end_color='D9E1F2', fill_type='solid')

### Select SQL 4

In [394]:
sql4 = file['sql 4']

d4 = pd.read_sql_query("""select 	
case when table_name = 'FUND' and contract_type ='IS2700'   then' Investment in FUND' else rating_model_new  end as rating_model_new_mapped,	
counterparty_new_type,counterparty_new_type_desc ,counterparty_new_code,counterparty_new_name,rating_model_new,booking_company,risk_country_new ,sum(ead) ead  ,sum(rwa) rwa 	
from rzb_cdr where (rating_model_new='Fund'  or (table_name = 'FUND' and contract_type ='IS2700' ) or (counterparty_new_type='57' and rating_model_new<>'Fund'))	
and  method_new = 'FOU' and NOT (table_name='EQUITY' and sub_method_new='STD')---to remove the grandfathering equities	
group by case when table_name = 'FUND' and contract_type ='IS2700'   then' Investment in FUND' else rating_model_new  end,	
counterparty_new_type,counterparty_new_type_desc ,counterparty_new_code,counterparty_new_name,rating_model_new,booking_company,risk_country_new""", connection_RAY)

### Input the values and add the colors

In [395]:
# add headers
for idx_c, header in enumerate(d4.columns.values):
    sql4.cell(row=1, column=idx_c+1).value = header
    sql4.cell(row=1, column=idx_c+1).fill = PatternFill(start_color='D9E1F2', end_color='D9E1F2', fill_type='solid')

# Iterate all rows using DataFrame.itertuples()
index = 0
for idx, row in enumerate(d4.itertuples(index = True)):
    for idx_c, header in enumerate(d4.columns.values):
        sql4.cell(row=idx+2, column=idx_c+1).value = getattr(row, header)
        index = idx_c

# Add the total        
sql4.cell(row=1, column=index+4).value = 'ead'
sql4.cell(row=1, column=index+5).value = 'rwa'

sql4.cell(row=2, column=index+3).value = 'check FUND FOU'
sql4.cell(row=2, column=index+4).value = "=SUM(I:I)-SUMIF('sql 1'!B:B,\"Fund\",'sql 1'!C:C)"
sql4.cell(row=2, column=index+5).value = "=SUM(J:J)-SUMIF('sql 1'!B:B,\"Fund\",'sql 1'!D:D)"

for col in range(3, 6, 1):
    sql4.cell(row=2, column=index+col).fill = PatternFill(start_color='D9E1F2', end_color='D9E1F2', fill_type='solid')

### Select SQL 6

In [396]:
sql6 = file['sql 6']

d6 = pd.read_sql_query("""select securitized, method_new,sum(ead_pre_ccf) ead_pre_ccf, sum(ead) ead, sum (rwa) rwa, sum(el) el 
from t_cdr
group by securitized,method_new""", connection_RAY)

### Input the values and add the colors

In [397]:
# add headers
for idx_c, header in enumerate(d6.columns.values):
    sql6.cell(row=1, column=idx_c+1).value = header
    sql6.cell(row=1, column=idx_c+1).fill = PatternFill(start_color='D9E1F2', end_color='D9E1F2', fill_type='solid')

# Iterate all rows using DataFrame.itertuples()
index = 0
for idx, row in enumerate(d6.itertuples(index = True)):
    for idx_c, header in enumerate(d6.columns.values):
        sql6.cell(row=idx+2, column=idx_c+1).value = getattr(row, header)
        index = idx

        
# Add the total        
sql6.cell(row=index + 3, column=2).value = 'Total'
sql6.cell(row=index + 3, column=3).value = '=SUM(C2:C'+ str(index+2) +')'
sql6.cell(row=index + 3, column=4).value = '=SUM(D2:D'+ str(index+2) +')'
sql6.cell(row=index + 3, column=5).value = '=SUM(E2:E'+ str(index+2) +')'
sql6.cell(row=index + 3, column=6).value = '=SUM(F2:F'+ str(index+2) +')'

sql6_total_end = index + 3

for col in range(2, 7, 1):
    sql6.cell(row=index+3, column=col).fill = PatternFill(start_color='D9E1F2', end_color='D9E1F2', fill_type='solid')

sql6.cell(row=index + 7, column=3).value = "T_CDR"
sql6.cell(row=index + 7, column=4).value = "=SUMIFS(D2:D" + str(index + 2) + ", B2:B" + str(index + 2) + ", \"FOU\", A2:A" + str(index + 2) + ", \"<>T\")"
sql6.cell(row=index + 7, column=5).value = "=SUMIFS(E2:E" + str(index + 2) + ", B2:B" + str(index + 2) + ", \"FOU\", A2:A" + str(index + 2) + ", \"<>T\")"


sql6.cell(row=index + 8, column=3).value = "data from rzb_Cdr sql 1"
sql6.cell(row=index + 8, column=4).value = "='sql 1'!C" + sql1_total
sql6.cell(row=index + 8, column=5).value = "='sql 1'!D" + sql1_total

sql6.cell(row=index + 9, column=3).value = "Check"
sql6.cell(row=index + 9, column=4).value = "=D" + str(index+7) + "-D" + str(index + 8)
sql6.cell(row=index + 9, column=5).value = "=E" + str(index+7) + "-E" + str(index + 8)


for col in range(3, 6, 1):
    sql6.cell(row=index+9, column=col).fill = PatternFill(start_color='D9E1F2', end_color='D9E1F2', fill_type='solid')

### Select SQL 5

In [398]:
sql5 = file['sql 5']

d5 = pd.read_sql_query("""select sum(ead_pre_ccf) ead_pre_ccf, sum(ead) ead, sum (rwa) rwa, sum(el) el from t_cdr""", connection_RAY)

In [399]:
# add headers
for idx_c, header in enumerate(d5.columns.values):
    sql5.cell(row=1, column=idx_c+1).value = header
    sql5.cell(row=1, column=idx_c+1).fill = PatternFill(start_color='D9E1F2', end_color='D9E1F2', fill_type='solid')

# Iterate all rows using DataFrame.itertuples()
index = 0
for idx, row in enumerate(d5.itertuples(index = True)):
    for idx_c, header in enumerate(d5.columns.values):
        sql5.cell(row=idx+2, column=idx_c+1).value = getattr(row, header)
        index = idx

        
# Add the total        
sql5.cell(row=index + 4, column=1).value = ' check  total t_Cdr'

sql5.cell(row=index + 5, column=1).value = "=A2-'sql 6'!C" + str(sql6_total_end)
sql5.cell(row=index + 5, column=2).value = "=B2-'sql 6'!D" + str(sql6_total_end)
sql5.cell(row=index + 5, column=3).value = "=C2-'sql 6'!E" + str(sql6_total_end)
sql5.cell(row=index + 5, column=4).value = "=D2-'sql 6'!F" + str(sql6_total_end)

for col in range(1, 5, 1):
    sql5.cell(row=index+5, column=col).fill = PatternFill(start_color='D9E1F2', end_color='D9E1F2', fill_type='solid')

### Select SQL 7

In [400]:
sql7 = file['sql 7']

d7 = pd.read_sql_query("""select 
case when table_name = 'FUND' and contract_type ='IS2700'   then 'Investment in FUND' else rating_model_new  end as rating_model_new_mapped
,rating_model_new,
case when (table_name='EQUITY' and sub_method_new='STD') then 'STD' else method_new end as method_new_mapped,
method_new,
sum (ead) ead ,sum (rwa) rwa, 
count(distinct counterparty_new_code) as number_of_observations, 
max(ead) as max_Ead,
min( ead) as min_ead,
median(ead) as median_ead,booking_company
from (
select rating_model_new,method_new,sub_method_new, booking_company, contract_type,table_name,counterparty_new_code,sum (ead) as ead,sum(rwa) as rwa
from rzb_cdr 
where record_source='T_CDR' and ead>0 --as requested by Zsolt
group by rating_model_new,method_new,sub_method_new,contract_type,table_name,counterparty_new_code, booking_company)
group by case when table_name = 'FUND' and contract_type ='IS2700'   then 'Investment in FUND' else rating_model_new  end, rating_model_new, booking_company,
case when (table_name='EQUITY' and sub_method_new='STD') then 'STD' else method_new end ,method_new, booking_company

""", connection_RAY)

### Inut the values and add the colors

In [401]:
# add headers
for idx_c, header in enumerate(d7.columns.values):
    sql7.cell(row=1, column=idx_c+1).value = header
    sql7.cell(row=1, column=idx_c+1).fill = PatternFill(start_color='D9E1F2', end_color='D9E1F2', fill_type='solid')

# Iterate all rows using DataFrame.itertuples()
index = 0
for idx, row in enumerate(d7.itertuples(index = True)):
    for idx_c, header in enumerate(d7.columns.values):
        sql7.cell(row=idx+2, column=idx_c+1).value = getattr(row, header)
        index = idx

index = index + 2
# Add the FOU        
sql7.cell(row=2, column=14).value = 'Total FOU'
sql7.cell(row=2, column=15).value = '=SUMIF(C2:C'+ str(index) +',"FOU",E2:E'+ str(index) +')'
sql7.cell(row=2, column=16).value = '=SUMIF(C2:C'+ str(index) +',"FOU",F2:F'+ str(index) +')'
sql7.cell(row=2, column=17).value = '=SUMIF(C2:C'+ str(index) +',"FOU",G2:G'+ str(index) +')'

# Add the STD        
sql7.cell(row=3, column=14).value = 'Total STD'
sql7.cell(row=3, column=15).value = '=SUMIF(C2:C'+ str(index) +',"STD",E2:E'+ str(index) +')'
sql7.cell(row=3, column=16).value = '=SUMIF(C2:C'+ str(index) +',"STD",F2:F'+ str(index) +')'
sql7.cell(row=3, column=17).value = '=SUMIF(C2:C'+ str(index) +',"STD",G2:G'+ str(index) +')'

# Add the ADV        
sql7.cell(row=4, column=14).value = 'Total ADV'
sql7.cell(row=4, column=15).value = '=SUMIF(C2:C'+ str(index) +',"ADV",E2:E'+ str(index) +')'
sql7.cell(row=4, column=16).value = '=SUMIF(C2:C'+ str(index) +',"ADV",F2:F'+ str(index) +')'
sql7.cell(row=4, column=17).value = '=SUMIF(C2:C'+ str(index) +',"ADV",G2:G'+ str(index) +')'

# Add the Total        
sql7.cell(row=5, column=14).value = 'Total'
sql7.cell(row=5, column=15).value = '=SUM(E2:E'+ str(index) +')'
sql7.cell(row=5, column=16).value = '=SUM(F2:F'+ str(index) +')'
sql7.cell(row=5, column=17).value = '=SUM(G2:G'+ str(index) +')'

###
sql7.cell(row=6, column=14).value = 'Check Total sum'
sql7.cell(row=6, column=15).value = '=O5-SUM(O2:O4)'
sql7.cell(row=6, column=16).value = '=P5-SUM(P2:P4)'
sql7.cell(row=6, column=17).value = '=Q5-SUM(Q2:Q4)'

# Add the check fou        
sql7.cell(row=7, column=14).value = 'check FOU'
sql7.cell(row=7, column=15).value = "=O2-'sql 1'!C" + str(sql1_total)
sql7.cell(row=7, column=16).value = "=P2-'sql 1'!D" + str(sql1_total)

# Add the check total        
sql7.cell(8, column=14).value = 'check total'
sql7.cell(8, column=15).value = "=O5-SUMIF('sql 6'!A2:A"+ str(sql6_total_end-1) +", \"<>T\",'sql 6'!D2:D" + str(sql6_total_end-1) + ")"
sql7.cell(8, column=16).value = "=P5-SUMIF('sql 6'!A2:A"+ str(sql6_total_end-1) +", \"<>T\",'sql 6'!E2:E" + str(sql6_total_end-1) + ")"

for col in range(14, 17, 1):
    sql7.cell(row=8, column=col).fill = PatternFill(start_color='7994D1', end_color='7994D1', fill_type='solid')
    sql7.cell(row=7, column=col).fill = PatternFill(start_color='7994D1', end_color='7994D1', fill_type='solid')

### Save the file

In [402]:
ex_folder = "Export/"
file.save(ex_folder + file_name)
file.close()